<a href="https://colab.research.google.com/github/dnr6419/BigData_study/blob/master/dacon_universe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
os.getcwd()
data = pd.read_csv("./train.csv")
data.head()
column_number = {}
cnt = 0
# 결측치 확인
data = data.dropna()

# type을 가변수 형태로 변환.
column_name = set(data['type'].values)
for i in column_name:
  column_number[i] = cnt
  cnt += 1
def to_number(x, dic):
    return dic[x]
data['type_num'] = data['type'].apply(lambda x: to_number(x, column_number)) 
print(len(set(data['type'].values)))

19


In [3]:
modified_data = data.drop(['id'],axis=1)
feature_columns = list(data.columns.difference(['type','type_num'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = data['type_num'] # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인


(159992, 22) (39999, 22) (159992,) (39999,)


In [4]:
# lightgbm으로 했을 때의 결과
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import time
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 20, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수 400
            'num_iterations' : 1000, # 반복횟수 3000
            'feature_fraction' : 0.8,
            'objective': 'multiclassova', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("lightgbm")
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


lightgbm
Accuracy: 87.37 %
Time: 438.68 seconds


In [5]:
# 다중 선형 회귀분석
import statsmodels.api as sm
# train_x, test_x, train_y, test_y
# 설명 변수 : train_x
# 종속 변수 : train_y

train_x_add = sm.add_constant(train_x,has_constant='add') # 상수항 추가
multi_model = sm.OLS(train_y,train_x_add)
#fitted_model = multi_model.fit()
#print(test_x.head())
# y_test_pred=fitted_model.predict(test_x)
# print("다중 선형 회귀분석")
# print("Accuracy: %.2f" % (accuracy_score(test_y, y_test_pred) * 100), "%") # 정확도 % 계산


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


        fiberID  fiberMag_g  fiberMag_i  ...   psfMag_r   psfMag_u   psfMag_z
166714      316   17.789047   17.472949  ...  17.216045  18.373946  17.050261
136824      270   17.513766   16.991773  ...  16.787686  18.083842  16.629452
76161       394   17.029283   17.221333  ...  16.763304  17.807731  16.931291
194333      566   17.130789   16.793492  ...  16.541837  17.625714  16.445633
100794      308   20.435935   20.517781  ...  20.092792  21.030438  19.447472

[5 rows x 22 columns]


In [6]:
# randomforest
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(n_estimators = 100, # 100번 추정
                                             max_depth = 20) # 트리 깊이
rf_model = random_forest_model.fit(train_x, train_y) # 학습 진행
rf_predict = rf_model.predict(test_x) # 평가 데이터 예측
print("랜덤 포레스트")
print("Accuracy: %.2f" % (accuracy_score(test_y, rf_predict) * 100), "%") # 정확도 % 계산


랜덤 포레스트
Accuracy: 87.68 %


In [10]:
# bagging_의사결정나무
from sklearn.ensemble import BaggingRegressor
bagging_decision_tree_model = BaggingRegressor(
                                                n_estimators = 100, # 100번 샘플링
                                                verbose = 1) # 학습 과정 표시
bagging_model = bagging_decision_tree_model.fit(train_x, train_y) # 학습 진행
predict3 = bagging_model.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, predict3) * 100), "%") # 정확도 % 계산

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s finished


ValueError: ignored